# Building a tool that takes a technical question, and responds with an explanation. This is a tool that you will be able to use yourself if needed.
# Demonstrating my Familiarity with OpenAI API, and also Ollama
# I would've done claude too, but ran out of API limit

# imports

In [35]:
import os
import requests
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import ollama
from IPython.display import display, HTML, Markdown
from datetime import datetime
from openai import OpenAI

# Declaring constants

In [9]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

# I'm setting up OpenAI (paid) and Ollama (Free, also runs locally on your machine no need for an API key)

# setting up OpenAI environment

In [10]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key retrieved")
else:
    print("There is a problem with your API key!! Check your key.")
    
MODEL = MODEL_GPT
openai = OpenAI()

API key retrieved


# Get gpt-4o-mini to answer, with streaming
# Stream gives you, Typewriter kind of effect while providing the output

In [19]:
response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "user", "content": question}
    ],
    stream=True
)

# print(response['message']['content'])  # This won't work with Streaming, so we use chunk!

# DO this instead:
print("GPT: ", end="")
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()  # New line when done
    

GPT: The provided code is a Python expression that uses a combination of a generator function and a set comprehension. Let's break it down:

### Explanation

1. **Set Comprehension**:
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This is a set comprehension that iterates over a collection called `books`.
   - For each `book` in `books`, it calls `book.get("author")` to retrieve the value of the "author" key.
   - The `if book.get("author")` clause ensures that only books that have a defined "author" (not `None` or an empty string) are considered.
   - The result is a set containing unique authors since sets inherently do not allow duplicate values.

2. **Yielding from Set**:
   ```python
   yield from ...
   ```
   - The `yield from` expression is used within a generator function. It delegates the iteration to the set created by the comprehension.
   - This means that instead of returning the entire set at once, it will yield each author one a

# Get gpt-4o-mini to answer, without streaming

In [42]:
# Response Template is standard for OpenAI API call
# Stream is False by Default
def GPT_no_stream(question, model):
    """
    Ask GPT a question with clean, simple text output
    """
    print("=" * 60)
    print(f"QUESTION: {question}")
    print("=" * 60)
    
    # Get the response
    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": question}],
        stream=False
    )
    
    answer = response.choices[0].message.content
    timestamp = datetime.now().strftime("%H:%M:%S")
    
    # Clean up the response text
    clean_answer = clean_text_output(answer)
    
    print(f"\nGPT RESPONSE ({timestamp}):")
    print("-" * 40)
    print(clean_answer)
    print("=" * 60)
    
    return clean_answer
# Calling the function
GPT_no_stream(question, MODEL_GPT)

QUESTION: 
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}


GPT RESPONSE (14:06:36):
----------------------------------------
The given code snippet you provided utilizes a generator and a set comprehension in Python. Here's a breakdown of what it does and why:

Breakdown of the Code

1. Set Comprehension:
- `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set containing the values retrieved from the "author" key in each dictionary `book` from the iterable `books`.
- `book.get("author")` tries to access the value associated with the key "author" in each `book`.
- The `if book.get("author")` condition ensures that only those books where an author exists (i.e., the value is not `None`, empty string, or any other falsy value) are included in the set.

2. Yield from:
- The `yield from` expression is used to yield all values from an iterable. In this case, it yields each a

'The given code snippet you provided utilizes a generator and a set comprehension in Python. Here\'s a breakdown of what it does and why:\n\nBreakdown of the Code\n\n1. Set Comprehension:\n- `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set containing the values retrieved from the "author" key in each dictionary `book` from the iterable `books`.\n- `book.get("author")` tries to access the value associated with the key "author" in each `book`.\n- The `if book.get("author")` condition ensures that only those books where an author exists (i.e., the value is not `None`, empty string, or any other falsy value) are included in the set.\n\n2. Yield from:\n- The `yield from` expression is used to yield all values from an iterable. In this case, it yields each author obtained from the set comprehension one at a time.\n- Using `yield from` allows this code to be used within a generator function, meaning that it can produce a sequence of autho

# setting up Ollama environment

In [7]:
# Check if Ollama service is running
try:
    # Test connection to Ollama
    models = ollama.list()
    print("Ollama service is running")
    
    # Check if llama3.2 model is available
    available_models = [model['name'] for model in models['models']]
    
    if 'llama3.2:latest' in available_models:
        print("Llama 3.2 model is available")
        MODEL = 'llama3.2'
    else:
        print("Llama 3.2 model not found! Run 'ollama pull llama3.2' in terminal")
        MODEL = None
        
except Exception as e:
    print(f"Problem connecting to Ollama: {e}")
    print("Make sure Ollama is installed and running")
    MODEL = None

Set up the client (Ollama doesn't need API keys - it runs locally!)
if MODEL_LLAMA:
    client = ollama
    print(f"Using model: {MODEL_LLAMA}")
else:
    print("Setup incomplete - fix the issues above first")

Using model: llama3.2


In [12]:
# Get Llama 3.2 to answer
response = ollama.chat(
    model= MODEL_LLAMA,
    messages=[
        {'role': 'user', 'content': question}
    ]
)

print(response['message']['content'])

This line of code is a generator expression that yields values from an iterable (in this case, the "books" list).

Here's a breakdown:

- `yield from`: This keyword is used to delegate the generation of values to another iterable. It essentially says "yield all the values from this iterable".

- `{book.get("author") for book in books if book.get("author")}`: This is an iterable expression that iterates over each item (`book`) in the `books` list.

  - The `for book in books` part iterates over each `book` in the list.
  
  - The `if book.get("author")` part filters out any `book` for which the value associated with the key `"author"` is not available (i.e., `None`). This ensures that only books with an author are considered.

- `yield from`: As mentioned earlier, this keyword delegates the generation of values to another iterable. So, when the previous expression yields values, these values will be passed on to the next iteration.

This code snippet does essentially the same thing as t